In [1]:
!nvidia-smi 

Tue Apr 12 12:49:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    38W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/iseg_2019/dataISEG_np/train_pt.zip", 'r') as zip_ref:
    zip_ref.extractall("data_train")
with zipfile.ZipFile("/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/iseg_2019/dataISEG_np/test_pt.zip", 'r') as zip_ref:
    zip_ref.extractall("data_test")
# with zipfile.ZipFile("/content/drive/MyDrive/iseg2019/dataISEG_np/pseudo_pt.zip", 'r') as zip_ref:
#     zip_ref.extractall("pseudo_data")

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/iseg_2019/"
%run Unet_iseg.ipynb
%run preprocessing_iseg.ipynb
%run "/content/drive/MyDrive/BRATS/convNext_3D.ipynb"

     |████████████████████████████████| 431 kB 4.3 MB/s 


In [ ]:
!pip install pytorch-lightning
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import gc
import glob 
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset, TensorDataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_pseudo.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "trainCSF_epoch", "trainGM_epoch",
                    "trainWM_epoch","val_loss","valCSF", "valGM","valWM"]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs) 
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock3D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [4]:
train_dataset = DataLoader(IsegLoader(), batch_size=32, pin_memory=True,
                        shuffle=True, num_workers=4, 
                        drop_last=True, prefetch_factor = 32)
test_dataset = DataLoader(IsegLoader(typeData="test"), batch_size=64,
                          num_workers=4, prefetch_factor=64)

In [6]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = Segnet()):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
        dice_csf, dice_gm, dice_wm = dice_CSF(y_true, y_pred), dice_GM( y_true, y_pred), dice_WM(y_true, y_pred)
        return loss, dice_csf, dice_gm, dice_wm

    def training_step(self, batch, batch_idx):
        loss, dice_csf, dice_gm, dice_wm = self._step(batch)
        metrics = {"loss": loss, "trainCSF": dice_csf, "trainGM": dice_gm, "trainWM": dice_wm}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_csf, dice_gm, dice_wm = self._step(batch)
        metrics = {"val_loss": loss, "valCSF": dice_csf, "valGM":dice_gm, "valWM": dice_wm}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_csf, dice_gm, dice_wm = self._step(batch)
        metrics = {"tset_loss": loss, "testCSF": dice_csf, "testGM":dice_gm, "testWM": dice_wm}
        self.log_dict(metrics, prog_bar = True)
        return metrics


    def configure_optimizers(self):
        optimizer = Nadam(self.model.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.7, patience=3, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "valWM"}

        return [optimizer]
    

In [ ]:
segmentor = Segmentor()
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("./weights/", filename="ckpt{valWM:0.4f}",
                                                            monitor="valWM", mode = "max",
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar()
logger = HistoryLogger()
swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=1)
PARAMS = {"gpus":1, "benchmark":True, "enable_progress_bar":True, "logger":False,
        #   "callbacks" : [progress_bar], 
        #   "callbacks" : [check_point, progress_bar, logger, swa], 
          "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":5,
          "precision":16,
           "overfit_batches" :1, 
          }

trainer = pl.Trainer(**PARAMS)
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/ckpt0.9775.ckpt")
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/current.ckpt")

In [ ]:
trainer.fit(segmentor, train_dataset, test_dataset)

In [ ]:
# for layer in segmentor.modules():
#     if isinstance(layer, DropBlock3D):
#         print(layer.drop_prob) 
# segmentor.configure_optimizers()

In [ ]:
# trainer.save_checkpoint("./weights/current.ckpt", weights_only=True)
# trainer.save_checkpoint("./weights/swa_pseudo.ckpt", weights_only=True)

In [ ]:
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/swa_pseudo.ckpt")

In [ ]:
trainer.test(segmentor, test_dataset, ckpt_path=None,)

In [ ]:
def predict(images, model, batch_size = 64):
    images = torch.from_numpy(images)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3), images.size(4)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
segmentor = segmentor.to(device)
segmentor.eval()
listOfT1_train = glob.glob("./dataISEG/iSeg-2019-Training/*T1.hdr")
for i in range(len(listOfT1_train)):
    print("\r read in: ", i+1,"th volume   ", end="")
    t1 = np.transpose(nib.load(listOfT1_train[i]).get_fdata(), (-1,0,1,2))
    t2 = np.transpose(nib.load(listOfT1_train[i].replace("T1", "T2")).get_fdata(), (-1,0,1,2))
    seg = np.transpose(nib.load(listOfT1_train[i].replace("T1", "label")).get_fdata(), (-1,0,1,2))
    t1 = min_max_preprocess(t1)
    t2 = min_max_preprocess(t2)
    
    t1 = np.asarray(t1, np.float32)
    t2 = np.asarray(t2, np.float32)
    seg = np.asarray(seg, np.int64)

    inputs = np.concatenate([t1, t2], axis= 0)
    del t1, t2
    input_patches = extract_patches(inputs, PATCH_SIZE, EXTRACTION_STEP)
    pred = predict(input_patches, segmentor)
    del input_patches
    torch.cuda.empty_cache()
    label_constructed = reconstruct_volume_avg(pred, inputs.shape[1:], EXTRACTION_STEP)
    del inputs, pred
    label_constructed = torch.from_numpy(label_constructed)
    seg =  torch.from_numpy(seg)
    for j in range(1,4):
        output_standard = torch.argmax(label_constructed, dim=0, keepdim = True)
        output_CSF = torch.where(output_standard == j, 1, 0)
        label_CSF = torch.where(seg == j, 1, 0)
        intersection = torch.sum(label_CSF * output_CSF)
        union = torch.sum(label_CSF) + torch.sum(output_CSF)
        dice = (2. * intersection) / (union)
        print(f"subject {i}: ", dice)
    del label_constructed, seg
    break


 read in:  1 th volume   

143it [03:09,  1.32s/it]


subject 0:  tensor(0.9954)
subject 0:  tensor(0.9905)
subject 0:  tensor(0.9881)


##pseudo

In [ ]:
def getValidIndex(label_constructed, patch_input, threshold):
    filter = np.max(label_constructed, axis=0,keepdims=True)
    patch = extract_patches(filter, (1, 32, 32, 32), EXTRACTION_STEP)
    valid_index = []
    for index in range(patch.shape[0]):
        if (not False in np.unique(patch[index] > threshold)) and np.sum(patch_input[index]) != 0:
            valid_index.append(index)
    return valid_index
# %cd /content/
# !mkdir pseudo_data

In [ ]:
# segmentor = segmentor.to(device)
# segmentor.eval()
# SAVE_PATH = "./pseudo_data/"
# listOfT1_train = glob.glob("./drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation/*T1.hdr")
# count = 0
# for i in range(len(listOfT1_train)):
#     print("\r read in: ", i+1,"th volume ", end="")
#     t1 = np.transpose(nib.load(listOfT1_train[i]).get_fdata(), (-1,0,1,2))
#     t2 = np.transpose(nib.load(listOfT1_train[i].replace("T1", "T2")).get_fdata(), (-1,0,1,2))
#     t1 = min_max_preprocess(t1)
#     t2 = min_max_preprocess(t2)
    
#     t1 = np.asarray(t1, np.float32)
#     t2 = np.asarray(t2, np.float32)
#     inputs = np.concatenate([t1, t2], axis= 0)
#     del t1, t2
#     input_patches = extract_patches(inputs, PATCH_SIZE, EXTRACTION_STEP)
#     pred = predict(input_patches, segmentor)
#     torch.cuda.empty_cache()
#     label_constructed = reconstruct_volume_avg(pred, inputs.shape[1:], EXTRACTION_STEP)
#     del pred

#     valid_index = getValidIndex(label_constructed, input_patches, 0.45)
#     data_input = input_patches[valid_index]
#     del input_patches, inputs

#     seg = np.argmax(label_constructed, axis=0)[np.newaxis]
#     label_patch = extract_patches(seg, (1,*PATCH_SIZE[1:]), EXTRACTION_STEP)
#     y = label_patch[valid_index]
#     del label_patch, seg, valid_index

#     for j in tqdm(range(data_input.shape[0])):   
#         np.savez_compressed(SAVE_PATH+"iseg_"+str(count), 
#                                 image=data_input[j], mask=y[j]) 
#         count += 1
# import shutil
# shutil.make_archive("/content/drive/MyDrive/iseg2019/dataISEG_np/pseudo_pt",
#                     "zip","/content/pseudo_data") 

## submit

In [ ]:
LABEL_SUBMIT_PATH = "./dataISEG/iSeg-2019-test-label/"
IMAGE_SUBMIT_PATH = "./dataISEG/iSeg-2019-Testing/"
segmentor = segmentor.to(device)
segmentor.eval()
for i in range(24, 40):
    print("\r read in: ", i,"th volume ", end="")
    t1 = np.transpose(nib.load(IMAGE_SUBMIT_PATH + f"subject-{i}-T1.hdr").get_fdata(), (-1,0,1,2))
    t2 = np.transpose(nib.load(IMAGE_SUBMIT_PATH + f"subject-{i}-T2.hdr").get_fdata(), (-1,0,1,2))
    t1 = min_max_preprocess(t1, 5, 95)
    t2 = min_max_preprocess(t2, 5, 99) 
    t1 = np.asarray(t1, np.float32)
    t2 = np.asarray(t2, np.float32)
    inputs = np.concatenate([t1, t2], axis= 0)
    del t1, t2
    input_patches = extract_patches(inputs, PATCH_SIZE, EXTRACTION_STEP)
    pred = predict(input_patches, segmentor, batch_size = 16)
    gc.collect()
    torch.cuda.empty_cache()
    label_constructed = reconstruct_volume_avg(pred, inputs.shape[1:], EXTRACTION_STEP)
    del pred, input_patches, inputs
    seg = np.argmax(label_constructed, axis=0)[..., np.newaxis].astype(np.uint8)
    affine = nib.load(IMAGE_SUBMIT_PATH + f"subject-{i}-T1.hdr").affine
    label_save = nib.Nifti1Image(seg, affine)
    nib.nifti1.save(label_save, LABEL_SUBMIT_PATH + f"subject-{i}-label.hdr")

    del affine, label_save, seg

 read in:  24 th volume 

1525it [09:47,  2.59it/s]


 read in:  25 th volume 

1525it [09:47,  2.59it/s]


 read in:  26 th volume 

...

771it [04:57,  2.59it/s]


 read in:  38 th volume 

771it [04:57,  2.59it/s]


 read in:  39 th volume 

771it [04:57,  2.59it/s]


In [ ]:
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

Tue Nov  2 06:57:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |   1326MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
i= 39
t1 = np.transpose(nib.load(IMAGE_SUBMIT_PATH + f"subject-{i}-T1.hdr").get_fdata(), (-1,0,1,2))
t2 = np.transpose(nib.load(IMAGE_SUBMIT_PATH + f"subject-{i}-T2.hdr").get_fdata(), (-1,0,1,2))
# t1 = min_max_preprocess(t1)
# t2 = min_max_preprocess(t2)
inputs = np.concatenate([t1, t2], axis= 0)
non_zeros = t1 > 0
low, high = np.percentile(t1[non_zeros], [5, 99])
print(low, high)

214.0 714.0


In [ ]:
a = nib.load("./dataISEG/iSeg-2019-test-label/subject-30-label.hdr").get_fdata()

In [ ]:
np.unique(seg)

array([0, 1, 2, 3])

In [ ]:
np.sum(np.where(seg == 3, 1, 0))

73685

In [ ]:
40329

In [ ]:
patches = inputs
seg_patches = seg
for i in range(80, 150):
    plt.figure(i+1, figsize=(12,12))
    plt.subplot(131),plt.imshow(patches[0,:,:,i]),plt.title('T1')
    plt.subplot(132),plt.imshow(patches[1,:,:,i]),plt.title('T2')
    plt.subplot(133),plt.imshow(seg_patches[:,:,i,0]),plt.title('seg')